# 2018-09-25 (martes)

## Encuesta docente

## Ejercicio Tarifa

--> Estructura "base" no lee toda la información necesaria. Ustedes deben escribir código para leer el resto de lineas.

## Problemas con Casen

--> Usar "Casen small"

--- 

## Exportar datos en Pandas

Hasta ahora hemos visto:

- cargar información en Pandas
    - Desde otras bases de datos (txt, csv, dta, xls)
    - Con listas de tuplas (webscraping)
- realizar ciertas manipulaciones
    - filtrar o agregar
    - crear nuevas variables
    - agregar más información ("merge" o "concat")
- comenzar a analizar la información 
    - tablas agregadas
    - gráficos

Hoy veremos cómo exportar datos, y cómo eso se relaciona con el flujo de análisis.


In [7]:
# Primero, cargamos algunos datos de prueba

import pandas as pd

data = [(2010, 1, 3), 
        (2010, 2, 6),
        (2010, 3, 8),
        (2010, 4, 1),
        (2011, 1, 4),
        (2011, 2, 3),
        (2011, 3, 7),
        (2011, 4, 70)]

df = pd.DataFrame(data, columns=['year', 'quarter', 'valor'])


In [8]:
df

,year,quarter,valor
0,2010,1,3
1,2010,2,6
2,2010,3,8
3,2010,4,1
4,2011,1,4
5,2011,2,3
6,2011,3,7
7,2011,4,70


### Exportar a CSV

In [14]:
df.to_csv('data.csv')

### Exportar a Excel

In [15]:
df.to_excel('data_excel.xlsx')

### Exportar a SQL

In [19]:
#En este caso, vamos a exportar a una base de datos SQL, usando el motor SQLite3:

import sqlite3

db_path = 'data_sql.db'

db_conn = sqlite3.connect(db_path)

df.to_sql('mis_datos', db_conn, if_exists='replace')



## SQL

"Structured Query Language", es un lenguaje creado para interactuar con bases de datos relacionales.

Principal instrucción: **SELECT**

```
SELECT
FROM 
WHERE
GROUP BY
```

Nota: existen multiples variaciones de este lenguaje, con pequeñas diferencias de sintaxis según el "motor" de bases de datos con el que estemos trabajando (Oracle, Postgres, MySQL, SQLite3, etc).


- SELECT: nos permite decirle al servidor de datos qué variables queremos extraer
- FROM: identifica el nombre de la tabla (o tablas) desde donde obtener los datos
- WHERE: sentencia opcional; nos permite filtrar las filas de la tabla señalada en "FROM" según algún criterio
- GROUP BY: sentencia opcional; nos permite agrupar los resultados por algún nivel de agrupación

### Ejemplos

Seleccione todas las filas con año igual a 2010

```
SELECT * 
FROM mis_datos 
WHERE year = 2010 ```


Seleccione el "valor" del último trimestre de cada año

```
SELECT year, valor 
FROM mis_datos 
WHERE quarter = 4
```


Seleccione el número de observaciones de la tabla

```
SELECT COUNT(*) 
FROM mis_datos
```

Seleccione el promedio de "valor" por año

```
SELECT YEAR, AVG(valor)
FROM mis_datos
GROUP BY year
```

### Ejercicios

Calcule el promedio de "valor" para cada trimestre.


SELECT QUARTER, AVG(VALOR)
FROM mis_datos
GROUP BY quarter

Seleccione las observaciones donde "valor" haya sido superior a 5.

select *
from mis_datos
where valor > 5


Calcule el promedio de "valor" para cada año, considerando sólo el "valor" de los dos primeros trimestres de cada año.


select year, avg(valor)
from mis_datos
where quarter = 1 or quarter = 2
group by year

Calcule el número de trimestres de cada año, donde "valor" haya sido superior a 5.

select year, count(*)
from mis_datos
where valor > 5
group by year

## Sentencia "JOIN"

La sentencia "JOIN" nos permite juntar información de tablas distintas (piensen en "merge").

Existen 5 tipos de JOIN:

- Cross join: producto cartesiano de dos tablas (todas las combinaciones posibles de filas)
- Inner join: "Intersección" entre dos tablas, según una "llave"
- Left outer join: "Intersección" + filas en A, según una "llave"
- Right outer join: "Intersección" + filas en B, según una "llave"
- Full outer join: "Intersección + filas en A + filas en B, según una "llave"

### Ejemplos

Antes de comenzar, agreguemos un poco más de informacion a la base de datos:


In [6]:
nueva_data = [(1, 'Primero'),
             (2, 'Segundo'),
             (3, 'Tercero'),
             (4, 'Cuarto')]

new_df = pd.DataFrame(nueva_data, columns=['quarter', 'nombre'])

new_df.to_sql('tabla_nombres', db_conn, if_exists='replace')


1. Calcule el promedio de "valor" para cada trimestre, usando el nombre del trimestre
```
SELECT nombre, AVG(valor)
FROM mis_datos
    INNER JOIN tabla_nombres ON mis_datos.quarter = tabla_nombres.quarter
GROUP BY nombre

```


1. Identifique los años/trimestres (con el "nombre" del trimestre) donde "valor" haya sido mayor a 10

```
SELECT year, nombre
FROM mis_datos
    INNER JOIN tabla_nombres ON mis_datos.quarter = tabla_nombres.quarter
WHERE valor > 5

```

### Ejemplo de lectura de base de datos en SQLite3

In [ ]:
import sqlite3
from IPython.display import display

#creamos una conexión para leer la base de datos
db_path = 'data_sql.db'
db_conn = sqlite3.connect(db_path)

#armamos un string, con la query que queremos correr
sql_query = '''
    SELECT *
    FROM mis_datos
    WHERE quarter = 4
'''

#ocupamos la función "read_sql", para leer la base de datos y simultáneamente cargar los datos a un DataFrame
new_data = pd.read_sql(sql_query, db_conn)

#mostramos la tabla obtenida
display(new_data)

In [ ]:
import sqlite3
from IPython.display import display

#creamos una conexión para leer la base de datos
db_path = 'data_sql.db'
db_conn = sqlite3.connect(db_path)

#armamos un string, con la query que queremos correr
sql_query = '''
    select *
    from mis_datos
    where year = 2010
'''

#ocupamos la función "read_sql", para leer la base de datos y simultáneamente cargar los datos a un DataFrame
%time new_data = pd.read_sql(sql_query, db_conn)

#mostramos la tabla obtenida
display(new_data)

In [ ]:

#armamos un string, con la query que queremos correr
sql_query = '''
    select AVG(valor) as promedio
    from mis_datos
'''

#ocupamos la función "read_sql", para leer la base de datos y simultáneamente cargar los datos a un DataFrame
new_data = pd.read_sql(sql_query, db_conn)

#mostramos la tabla obtenida
display(new_data)

In [2]:
import pandas as pd
import sqlite3

#leo los datos
db_path = 'data_sql.db'
db_conn = sqlite3.connect(db_path)


#armamos un string, con la query que queremos correr
sql_query = '''
SELECT year, nombre
FROM mis_datos
    INNER JOIN tabla_nombres ON mis_datos.quarter = tabla_nombres.quarter
WHERE valor > 5
'''

#ocupamos la función "read_sql", para leer la base de datos y simultáneamente cargar los datos a un DataFrame
new_data = pd.read_sql(sql_query, db_conn)

#mostramos la tabla obtenida
display(new_data)



,year,nombre
0,2010,Segundo
1,2010,Tercero
2,2011,Tercero
3,2011,Cuarto


In [17]:
import pandas as pd
import sqlite3

#leo los datos
db_path = 'data_sql.db'
db_conn = sqlite3.connect(db_path)


#armamos un string, con la query que queremos correr
sql_query = '''
SELECT nombre, AVG(valor)
FROM mis_datos
    INNER JOIN tabla_nombres ON mis_datos.quarter = tabla_nombres.quarter
GROUP BY nombre
'''

#ocupamos la función "read_sql", para leer la base de datos y simultáneamente cargar los datos a un DataFrame
new_data = pd.read_sql(sql_query, db_conn)

#mostramos la tabla obtenida
display(new_data)



,nombre,AVG(valor)
0,Cuarto,35.5
1,Primero,3.5
2,Segundo,4.5
3,Tercero,7.5


In [16]:
new_data.quarter

0    1
1    2
2    3
3    4
Name: quarter, dtype: int64